I just apply the [solution](https://www.kaggle.com/code/hirotakanogami/h-m-eda-customer-clustering-by-kmeans) to this [notebook](https://www.kaggle.com/code/baekseungyun/lb-0-0235-ensemble-gives-you-bronze-medal). Thanks.

In [1]:
import os
import numpy as np
import pandas as pd
import gc

# To ensemble I used submissions from 7 public notebooks:
* LB: 0.0262 - my_submission, lightGBM
* LB: 0.0262 - my_submission, XGBoost
* LB: 0.0260 - my_submission, CatBoost
* LB: 0.0240 - ensamble model

In [2]:
sub0 = pd.read_csv('../input/mysubmission2/submission.csv').sort_values('customer_id').reset_index(drop=True) # 0.0262
sub1 = pd.read_csv('../input/xgboost/submission_xgb.csv').sort_values('customer_id').reset_index(drop=True) # 0.0262
sub2 = pd.read_csv('../input/catboost/submission_cat.csv').sort_values('customer_id').reset_index(drop=True) # 0.0260
sub3 = pd.read_csv('../input/lb-0-0240-h-m-ensemble-magic-multi-blend/submission.csv').sort_values('customer_id').reset_index(drop=True) # 0.0240

In [3]:
sub0.columns = ['customer_id', 'prediction0']
sub0['prediction1'] = sub1['prediction']
sub0['prediction2'] = sub2['prediction']
sub0['prediction3'] = sub3['prediction']

del sub1, sub2, sub3
gc.collect()
sub0.head()

,customer_id,prediction0,prediction1,prediction2,prediction3
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601006 0568601043 0924243001 0919273002 09...,0568601006 0568601043 0858856005 0924243001 09...,0568601006 0568601043 0924243001 0915529003 07...,0568601043 0568601006 0751471001 0924243001 04...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0673677002 0915529005 0924243002 0896169005 09...,0924243001 0915529005 0923758001 0918522001 09...,0924243001 0924243002 0915529005 0918522001 09...,0826211002 0924243001 0924243002 0739590027 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0805000001 0794321011 0924243001 09...,0794321007 0805000001 0794321011 0924243001 09...,0805000001 0794321007 0924243001 0924243002 09...,0794321007 0852643001 0924243001 0924243002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0751471001 0751471043 0720125001 0919273002 09...,0923758001 0915529003 0714790020 0751471043 09...,0923758001 0751471001 0924243002 0915529003 09...,0448509014 0924243002 0924243001 0751471001 05...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0896152002 0896152001 09...,0730683050 0791587015 0918522001 0915529003 09...,0730683050 0896152002 0791587015 0924243001 07...,0730683050 0791587015 0927530004 0896152002 09...


In [4]:
def cust_blend(dt, W = [1,1,1,1,1]):
    #Global ensemble weights
    #W = [1.15,0.95,0.85]

    #Create a list of all model predictions
    REC = []

    # Second Try
    REC.append(dt['prediction0'].split())
    REC.append(dt['prediction1'].split())
    REC.append(dt['prediction2'].split())
    REC.append(dt['prediction3'].split())

    #Create a dictionary of items recommended.
    #Assign a weight according the order of appearance and multiply by global weights
    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))

    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())

    # Return the top 12 items only
    return ' '.join(res[:12])


sub0['prediction'] = sub0.apply(cust_blend, W = [1.0, 1.2, 0.9, 0.3], axis=1)
sub0.head()

,customer_id,prediction0,prediction1,prediction2,prediction3,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601006 0568601043 0924243001 0919273002 09...,0568601006 0568601043 0858856005 0924243001 09...,0568601006 0568601043 0924243001 0915529003 07...,0568601043 0568601006 0751471001 0924243001 04...,0568601006 0568601043 0924243001 0915529003 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0673677002 0915529005 0924243002 0896169005 09...,0924243001 0915529005 0923758001 0918522001 09...,0924243001 0924243002 0915529005 0918522001 09...,0826211002 0924243001 0924243002 0739590027 08...,0924243001 0915529005 0673677002 0924243002 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0805000001 0794321011 0924243001 09...,0794321007 0805000001 0794321011 0924243001 09...,0805000001 0794321007 0924243001 0924243002 09...,0794321007 0852643001 0924243001 0924243002 07...,0794321007 0805000001 0924243001 0794321011 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0751471001 0751471043 0720125001 0919273002 09...,0923758001 0915529003 0714790020 0751471043 09...,0923758001 0751471001 0924243002 0915529003 09...,0448509014 0924243002 0924243001 0751471001 05...,0923758001 0751471001 0915529003 0751471043 09...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0896152002 0896152001 09...,0730683050 0791587015 0918522001 0915529003 09...,0730683050 0896152002 0791587015 0924243001 07...,0730683050 0791587015 0927530004 0896152002 09...,0730683050 0791587015 0896152002 0924243001 09...


# Make a submission

In [5]:
del sub0['prediction0']
del sub0['prediction1']
del sub0['prediction2']
del sub0['prediction3']

gc.collect()


sub0.to_csv('submission.csv', index=False)